In [1]:
# -*- coding: utf-8 -*-
# @Author: Kun
import requests 
from lxml import etree
import pandas as pd

df = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4343.0 Safari/537.36',
           'Referer': 'https://movie.douban.com/top250'}
columns = ['排名','电影名称','导演','上映年份','制作国家','类型','评分','评价人数','短评']

In [2]:
def get_data(html):
    xp = etree.HTML(html)
    lis = xp.xpath('//*[@id="content"]/div/div[1]/ol/li')## 找到影片信息
    for li in lis:
        """排名、标题、导演、演员、"""
        ranks = li.xpath('div/div[1]/em/text()')##排名
        titles = li.xpath('div/div[2]/div[1]/a/span[1]/text()')##标题名
        directors = li.xpath('div/div[2]/div[2]/p[1]/text()')[0].strip().replace("\xa0\xa0\xa0","\t").split("\t")
        infos = li.xpath('div/div[2]/div[2]/p[1]/text()')[1].strip().replace('\xa0','').split('/')##消\xa0
        dates,areas,genres = infos[0],infos[1],infos[2]##年份，地区，标签
        ratings = li.xpath('.//div[@class="star"]/span[2]/text()')[0]##评分
        scores = li.xpath('.//div[@class="star"]/span[4]/text()')[0][:-3]##评分人数
        quotes = li.xpath('.//p[@class="quote"]/span/text()')##短句
        for rank,title,director in zip(ranks,titles,directors):
            if len(quotes) == 0:
                quotes = None
            else:
                quotes = quotes[0]
            df.append([rank,title,director,dates,areas,genres,ratings,scores,quotes])


In [3]:
for i in range(0,251,25):
    url = "https://movie.douban.com/top250?start={}&filter=".format(str(i))
    res = requests.get(url,headers=headers)
    html = res.text
    get_data(html)

In [4]:
columns

['排名', '电影名称', '导演', '上映年份', '制作国家', '类型', '评分', '评价人数', '短评']

In [5]:
d = pd.DataFrame(df,columns=columns)
d.to_csv('Top2500.csv',index=False)

# 将数据导入MySQL

In [8]:

import pymysql.cursors
import pandas as pd
 
conn= pymysql.connect(host='localhost',port=3306,user='root',passwd='123456',
                             charset='utf8',
                             cursorclass=pymysql.cursors.DictCursor)

pd.to_xlsx(conn)


In [69]:
df=pd.read_excel("a.xlsx")
#index =false 是不展示dataframe的index列  if_exists ‘append’代表 表存在即追加，‘replace’代表重建
df.iloc[:,:].to_sql('texts',conn,flavor='mysql',if_exists='append',index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'a.xlsx'